# Projet INSEE - MOBILITÉ TRANSFRONTALIÈRE

In [7]:
import pandas as pd
import numpy as np

# ============================================================
# ÉTAPE 1 — CHARGEMENT ROBUSTE DE LA BDD (ANTI-PIÈGES INSEE)
# ============================================================

INPUT_PATH = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/BDD_FINAL_CLEAN.csv"

print("\n[1] TEST DU FORMAT DU FICHIER")
with open(INPUT_PATH, "r", encoding="utf-8", errors="ignore") as f:
    head = f.readline()

print("Première ligne (extrait) :")
print(head[:300])
print("';' :", head.count(";"), " | ',' :", head.count(","), " | '\\t' :", head.count("\t"))

print("\n[2] LECTURE ROBUSTE DU CSV")
df = pd.read_csv(
    INPUT_PATH,
    sep=None,            # auto-détection
    engine="python",
    encoding="utf-8",
)

print("✔ Dimensions :", df.shape)
print("✔ Colonnes (20 premières) :", list(df.columns[:20]))

# ============================================================
# VERROUILLAGE DES VARIABLES CLÉS
# ============================================================

# Sécurité noms de colonnes
if "DCFLT" not in df.columns and "DCLT" in df.columns:
    df = df.rename(columns={"DCLT": "DCFLT"})

# Typage strict
df["ILT"] = pd.to_numeric(df["ILT"], errors="coerce").astype("Int64")
df["DCFLT"] = df["DCFLT"].astype("string")

print("\n[3] VÉRIFICATIONS CRITIQUES")
print("ILT présent :", "ILT" in df.columns)
print("DCFLT présent :", "DCFLT" in df.columns)
print("Distribution ILT :")
print(df["ILT"].value_counts(dropna=False).sort_index())



[1] TEST DU FORMAT DU FICHIER
Première ligne (extrait) :
COMMUNE;ARM;DCFLT;DCLT;AGEREVQ;GS;DEROU;DIPL;EMPL;ILT;ILTUU;IMMI;INATC;INEEM;INPOM;INPSM;IPONDI;LPRM;METRODOM;MOCO;NA5;NPERR;REGION;REGLT;SEXE;STAT;STOCD;TP;TRANS;TYPL;TYPMR;VOIT;DEPT;DEPCOM;CANTVILLE;is_unique_key;NUMMI;ACHLR;AEMMR;AGED;AGER20;AGEREV;ANAI;ANEMR;APAF;ASCEN;BAIN;BATI;CATIRIS;CATL;CAT
';' : 105  | ',' : 0  | '\t' : 0

[2] LECTURE ROBUSTE DU CSV
✔ Dimensions : (494483, 106)
✔ Colonnes (20 premières) : ['COMMUNE', 'ARM', 'DCFLT', 'DCLT', 'AGEREVQ', 'GS', 'DEROU', 'DIPL', 'EMPL', 'ILT', 'ILTUU', 'IMMI', 'INATC', 'INEEM', 'INPOM', 'INPSM', 'IPONDI', 'LPRM', 'METRODOM', 'MOCO']

[3] VÉRIFICATIONS CRITIQUES
ILT présent : True
DCFLT présent : True
Distribution ILT :
ILT
1    158593
2    253635
3     27098
4     10840
5        46
6         7
7     44264
Name: count, dtype: Int64


# Création de variables 

In [14]:
import pandas as pd
import numpy as np

# ============================================================
# PARAMÈTRES (adapte tes chemins)
# ============================================================
INPUT_PATH = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/BDD_FINAL_CLEAN.csv"
OUTPUT_CSV_PATH = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/BDD_FINAL_new.csv"
OUTPUT_XLSX_SAMPLE_PATH = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/BDD_FINAL_new_sample10k.xlsx"

SAMPLE_N = 10_000
RANDOM_STATE = 42

# ============================================================
# 1) CHARGEMENT (ton fichier est en ; -> obligatoire)
# ============================================================
print("== Chargement ==")
df = pd.read_csv(INPUT_PATH, sep=";", encoding="utf-8", low_memory=False)
print(f"OK: {df.shape[0]:,} lignes, {df.shape[1]} colonnes")

# Sécurités
if "DCFLT" not in df.columns and "DCLT" in df.columns:
    df = df.rename(columns={"DCLT": "DCFLT"})

for col in ["ILT", "DCFLT"]:
    if col not in df.columns:
        raise ValueError(f"Colonne manquante: {col}")

# Types propres
df["ILT"] = pd.to_numeric(df["ILT"], errors="coerce").astype("Int64")
df["DCFLT"] = df["DCFLT"].astype("string").str.strip().str.upper()

# ============================================================
# 2) ID
# ============================================================
print("== ID_INDIVIDU ==")
df["ID_INDIVIDU"] = np.arange(1, len(df) + 1, dtype=np.int32)
print("OK: ID uniques =", df["ID_INDIVIDU"].is_unique)

# ============================================================
# 3) TRANSFRONTALIER
# ============================================================
print("== TRANSFRONTALIER ==")
df["TRANSFRONTALIER"] = (df["ILT"] == 7).astype("int8")
print("OK: transfrontaliers =", int(df["TRANSFRONTALIER"].sum()))

# ============================================================
# 4) WORK_IN_PAYS (vectorisé, rapide)
# ============================================================
print("== WORK_IN_PAYS ==")

dc = df["DCFLT"]

# Par défaut
work = np.full(len(df), "Autre_etranger_ou_inconnu", dtype=object)

# France
mask_fr = (dc == "99999")
work[mask_fr] = "France"

# Préfixes
work[dc.str.startswith("LU", na=False)] = "Luxembourg"
work[dc.str.startswith("AL", na=False)] = "Allemagne"
work[dc.str.startswith("SU", na=False)] = "Suisse"
work[dc.str.startswith("BE", na=False)] = "Belgique"
work[dc.str.startswith("MO", na=False)] = "Monaco"

df["WORK_IN_PAYS"] = work

# IMPORTANT: pour éviter les faux positifs, on force les non-transfrontaliers à France
df.loc[df["TRANSFRONTALIER"] == 0, "WORK_IN_PAYS"] = "France"

# ============================================================
# 5) DUMMIES PAYS
# ============================================================
print("== Dummies pays ==")
df["WORK_LUXEMBOURG"] = (df["WORK_IN_PAYS"] == "Luxembourg").astype("int8")
df["WORK_ALLEMAGNE"]  = (df["WORK_IN_PAYS"] == "Allemagne").astype("int8")
df["WORK_SUISSE"]     = (df["WORK_IN_PAYS"] == "Suisse").astype("int8")
df["WORK_BELGIQUE"]   = (df["WORK_IN_PAYS"] == "Belgique").astype("int8")

# ============================================================
# 6) CHECKS (peu de prints, pas de spam)
# ============================================================
print("== Checks cohérence ==")
incoh_ilt7_fr = df[(df["TRANSFRONTALIER"] == 1) & (df["DCFLT"] == "99999")]
print("ILT=7 mais DCFLT=99999 :", len(incoh_ilt7_fr))

print("Répartition WORK_IN_PAYS (transfrontaliers uniquement) :")
print(df.loc[df["TRANSFRONTALIER"] == 1, "WORK_IN_PAYS"].value_counts())

# ============================================================
# 7) EXPORT CSV enrichi (en ; pour rester cohérent Excel/INSEE)
# ============================================================
print("== Export CSV ==")
df.to_csv(OUTPUT_CSV_PATH, index=False, sep=";")
print("OK:", OUTPUT_CSV_PATH)

# ============================================================
# 8) EXPORT EXCEL SAMPLE 10 000 lignes
# ============================================================
print("== Export Excel sample 10k ==")
sample_df = df.sample(n=min(SAMPLE_N, len(df)), random_state=RANDOM_STATE).copy()

# Colonnes à mettre devant pour vérif rapide
front = ["ID_INDIVIDU", "ILT", "DCFLT", "TRANSFRONTALIER", "WORK_IN_PAYS",
         "WORK_LUXEMBOURG", "WORK_ALLEMAGNE", "WORK_SUISSE", "WORK_BELGIQUE",
         "REGLT", "DEPT", "DEPCOM", "CANTVILLE"]
front = [c for c in front if c in sample_df.columns]
sample_df = sample_df[front + [c for c in sample_df.columns if c not in front]]

sample_df.to_excel(OUTPUT_XLSX_SAMPLE_PATH, index=False)
print("OK:", OUTPUT_XLSX_SAMPLE_PATH)

print("== Terminé ==")


== Chargement ==
OK: 494,483 lignes, 106 colonnes
== ID_INDIVIDU ==
OK: ID uniques = True
== TRANSFRONTALIER ==
OK: transfrontaliers = 44264
== WORK_IN_PAYS ==
== Dummies pays ==
== Checks cohérence ==
ILT=7 mais DCFLT=99999 : 0
Répartition WORK_IN_PAYS (transfrontaliers uniquement) :
WORK_IN_PAYS
Luxembourg                   23563
Allemagne                    10135
Suisse                        9407
Belgique                       965
Autre_etranger_ou_inconnu      188
Monaco                           6
Name: count, dtype: int64
== Export CSV ==
OK: /Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/BDD_FINAL_new.csv
== Export Excel sample 10k ==


Exception ignored in: <function ZipFile.__del__ at 0x110c8ea20>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/zipfile/__init__.py", line 1940, in __del__
  File "/opt/anaconda3/lib/python3.12/zipfile/__init__.py", line 1957, in close
ValueError: seek of closed file


OK: /Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/BDD_FINAL_new_sample10k.xlsx
== Terminé ==


## Stratégie d’estimation (niveau 1) : Probit et sélection de variables assistée par Machine Learning

### Objectif du premier volet
Le premier volet de ce travail vise à estimer un **modèle probit binaire** afin d’analyser les déterminants de la probabilité pour un individu d’être **travailleur transfrontalier dans la région Grand Est**. La variable dépendante prend la valeur 1 lorsque l’individu exerce une activité professionnelle à l’étranger tout en résidant dans le Grand Est, et 0 sinon.

Le choix du probit s’inscrit dans un cadre économétrique standard pour les variables dichotomiques. Il permet une interprétation structurée des résultats, notamment à travers les **effets marginaux**, et constitue une référence largement utilisée dans l’analyse des comportements de mobilité du travail.

### Pourquoi compléter le probit par une étape de Machine Learning ?
La base de données finale résulte de l’appariement de sources INSEE détaillées (INDCVI et MOBPRO) et comprend un nombre important de variables individuelles, professionnelles, géographiques et liées au logement. Dans ce contexte, la sélection des covariables du modèle probit pose plusieurs enjeux :

- limitation de l’**arbitraire** dans le choix des variables explicatives,
- gestion de la **multicolinéarité** entre caractéristiques socio-démographiques et professionnelles,
- identification de variables prédictives pertinentes dans un espace de dimension élevée.

L’introduction d’une étape de Machine Learning ne vise pas à remplacer l’approche économétrique, mais à **sécuriser la spécification du modèle probit** en apportant une procédure de sélection systématique, reproductible et guidée par les données.

### Méthode retenue : sélection pénalisée de type LASSO
Une régression binaire pénalisée de type **LASSO (L1)** est utilisée comme outil de présélection des covariables. Cette méthode permet :

- une **sélection automatique** des variables les plus informatives pour la prédiction du statut de travailleur transfrontalier,
- la réduction de la dimension du modèle en mettant à zéro les coefficients des variables peu contributives,
- une meilleure gestion des corrélations entre variables issues de sources riches et détaillées,
- un calibrage rigoureux via **validation croisée**, limitant les risques de sur-ajustement.

À ce stade exploratoire, l’utilisation d’un lien logit (logit-LASSO) est retenue pour des raisons pratiques et de robustesse numérique. La différence entre logit et probit portant essentiellement sur la forme de la fonction de lien, cette étape est considérée comme neutre du point de vue de la sélection des variables, l’objectif n’étant pas l’interprétation des coefficients mais l’identification des covariables informatives.

### Enchaînement des étapes empiriques
La stratégie d’estimation repose ainsi sur une approche en deux niveaux :

1. **Sélection assistée par Machine Learning (outil auxiliaire)**  
   - Estimation d’un modèle binaire pénalisé (LASSO) sur un échantillon de travail.
   - Identification d’un ensemble restreint de variables candidates expliquant la probabilité d’être travailleur transfrontalier dans le Grand Est.

2. **Estimation économétrique (modèle principal)**  
   - Estimation d’un **probit standard** par maximum de vraisemblance sur la base finale.
   - Analyse des coefficients et des effets marginaux afin de caractériser les déterminants individuels, professionnels et géographiques de la mobilité transfrontalière.

### Positionnement méthodologique
Cette approche permet de concilier :
- la **rigueur interprétable** du modèle probit, au cœur de l’analyse économique,
- et une sélection de variables **robuste et peu arbitraire**, issue d’outils de Machine Learning utilisés comme support méthodologique.

L’étape de Machine Learning est ainsi mobilisée comme un **complément à l’économétrie**, destiné à renforcer la solidité et la crédibilité de l’analyse des travailleurs transfrontaliers dans la région Grand Est.


## Choix de la méthode de pénalisation pour la sélection des variables

Dans le cadre de l’estimation d’un modèle probit visant à analyser les déterminants du travail transfrontalier dans la région Grand Est, une étape préalable de sélection des covariables est nécessaire afin de construire une spécification parcimonieuse et interprétable.

Plusieurs méthodes de pénalisation sont envisageables :

- **Ridge (L2)** : cette méthode réduit la variance des estimateurs en pénalisant la taille des coefficients, mais **ne réalise pas de sélection explicite** des variables. L’ensemble des covariables demeure dans le modèle, ce qui ne répond pas à l’objectif de construction d’une spécification économétrique claire.

- **Elastic Net (L1 + L2)** : cette approche combine les propriétés du LASSO et du Ridge et est particulièrement adaptée lorsque les variables explicatives sont fortement corrélées par groupes. Bien que pertinente, elle tend à conserver des ensembles de variables plus larges et conduit à des modèles moins parcimonieux.

- **LASSO (L1)** : le LASSO permet une **sélection automatique et explicite** des variables en annulant les coefficients des covariables peu informatives. Il constitue ainsi un outil efficace de réduction de dimension et de présélection des variables pertinentes.

Compte tenu de l’objectif principal — **construire un modèle probit interprétable pour l’analyse des travailleurs transfrontaliers du Grand Est** — le **LASSO est retenu** comme méthode de sélection des covariables. Cette approche offre un compromis adapté entre rigueur méthodologique, transparence de la spécification et facilité d’intégration dans un cadre économétrique standard.


In [6]:
"""
PROFILING COMPLET DE LA BDD (variables, types, manquants, modalités, stats num)
- Lit BDD_FINAL_new.csv (séparateur ; )
- Produit un tableau "profile" (1 ligne par variable)
- Exporte un Excel avec 2 onglets :
    1) profil_variables : complet
    2) resume : plus lisible (colonnes clés + min/p50/max)
- Affiche aussi des résumés utiles dans la console

Auteur : Projet INSEE - Mobilité Transfrontalière (Mehdi)
Date   : Janvier 2026
"""

import os
import numpy as np
import pandas as pd
from pandas.api.types import (
    is_numeric_dtype,
    is_bool_dtype,
    is_datetime64_any_dtype
)

# =============================================================================
# PARAMÈTRES (adapte si besoin)
# =============================================================================
INPUT_PATH = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/BDD_FINAL_new.csv"

OUT_PROFILE_XLSX = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/PROFILE_BDD_FINAL_new.xlsx"
OUT_PROFILE_CSV  = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/PROFILE_BDD_FINAL_new.csv"

TOP_K_MODALITES = 20          # top modalités pour catégorielles
SAMPLE_ROWS_FOR_MODALITES = None  # ex: 200_000 si tu veux accélérer sur une grosse base (None = tout)

# =============================================================================
# UTILS
# =============================================================================
def safe_nanpercentile(arr, q):
    """
    Percentiles robustes (gère tableaux vides / bool / non-num).
    """
    arr = np.asarray(arr)
    # si vide -> nan
    if arr.size == 0:
        return np.nan
    # retire NaN
    arr = arr[~np.isnan(arr)]
    if arr.size == 0:
        return np.nan
    return float(np.nanpercentile(arr.astype(float), q))


def simplify_type(series: pd.Series) -> str:
    """
    Renvoie un type simplifié lisible.
    """
    if is_bool_dtype(series):
        return "bool"
    if is_datetime64_any_dtype(series):
        return "datetime"
    if is_numeric_dtype(series):
        return "numeric"
    return "categorical"


def top_modalities(series: pd.Series, k=20) -> str:
    """
    Renvoie une string "modalite (count), ..." pour les catégorielles.
    Pour éviter l'explosion mémoire, on peut échantillonner.
    """
    s = series

    if SAMPLE_ROWS_FOR_MODALITES is not None and len(s) > SAMPLE_ROWS_FOR_MODALITES:
        s = s.sample(SAMPLE_ROWS_FOR_MODALITES, random_state=42)

    # Normalise un peu pour la lisibilité
    # (on garde tel quel, juste strip pour string)
    if s.dtype == "object" or str(s.dtype).startswith("string"):
        s = s.astype("string").str.strip()

    vc = s.value_counts(dropna=False).head(k)
    parts = []
    for idx, cnt in vc.items():
        if pd.isna(idx):
            label = "NA"
        else:
            label = str(idx)
        parts.append(f"{label} ({int(cnt)})")
    return ", ".join(parts)


# =============================================================================
# 1) CHARGEMENT
# =============================================================================
print("=" * 90)
print("PROFILING BDD_FINAL_new.csv")
print("=" * 90)

if not os.path.exists(INPUT_PATH):
    raise FileNotFoundError(f"Fichier introuvable: {INPUT_PATH}")

print(f"[1] Lecture : {INPUT_PATH}")
df = pd.read_csv(INPUT_PATH, sep=";", encoding="utf-8", low_memory=False)

print(f"OK: {df.shape[0]:,} lignes | {df.shape[1]} colonnes")
print("Aperçu colonnes (20 premières):", list(df.columns[:20]))

# =============================================================================
# 2) PROFILING COLONNE PAR COLONNE
# =============================================================================
print("\n[2] Construction du profil variables…")

rows = []
n = len(df)

for col in df.columns:
    s = df[col]
    dtype = str(s.dtype)

    # manquants
    na_count = int(s.isna().sum())
    na_pct = (na_count / n) * 100 if n > 0 else np.nan

    # type simplifié
    t_simpl = simplify_type(s)

    # nb "modalités" / uniques (on garde la même colonne pour tout)
    n_unique = int(s.nunique(dropna=True))

    # stats numériques (si numérique ou bool)
    stats = {"min": "", "p1": "", "p50": "", "p99": "", "max": ""}

    if t_simpl in ("numeric", "bool"):
        # IMPORTANT: booleans -> cast int, sinon numpy peut planter sur quantiles
        if is_bool_dtype(s):
            s_num = s.astype("Int64").astype(float)
        else:
            s_num = pd.to_numeric(s, errors="coerce").astype(float)

        arr = s_num.values
        # retire NaN pour checks
        arr_nonan = arr[np.isfinite(arr)]

        if arr_nonan.size > 0:
            stats = {
                "min": float(np.nanmin(arr_nonan)),
                "p1":  safe_nanpercentile(arr_nonan, 1),
                "p50": safe_nanpercentile(arr_nonan, 50),
                "p99": safe_nanpercentile(arr_nonan, 99),
                "max": float(np.nanmax(arr_nonan)),
            }
        else:
            stats = {"min": np.nan, "p1": np.nan, "p50": np.nan, "p99": np.nan, "max": np.nan}

    # modalités principales (si catégorielle, sinon vide)
    mod_princ = ""
    if t_simpl == "categorical":
        mod_princ = top_modalities(s, k=TOP_K_MODALITES)

    rows.append({
        "variable": col,
        "dtype_pandas": dtype,
        "type_simplifie": t_simpl,
        "nb_modalites_ou_valeurs_uniques": n_unique,
        "taux_manquants_%": round(na_pct, 4),
        "nb_manquants": na_count,
        "modalites_principales": mod_princ,
        "min": stats["min"],
        "p1": stats["p1"],
        "p50": stats["p50"],
        "p99": stats["p99"],
        "max": stats["max"],
    })

profile = pd.DataFrame(rows)

# Tri utile: d’abord manquants décroissants, puis cardinalité décroissante
profile = profile.sort_values(
    by=["taux_manquants_%", "nb_modalites_ou_valeurs_uniques"],
    ascending=[False, False]
).reset_index(drop=True)

print("OK: profil construit")
print(f"→ {profile.shape[0]} variables profilées")

# =============================================================================
# 3) RÉSUMÉS CONSOLE (lisibles, pas de spam)
# =============================================================================
print("\n" + "=" * 90)
print("RÉSUMÉS (console)")
print("=" * 90)

print("\nTop 15 variables avec le plus de manquants :")
print(
    profile[["variable", "dtype_pandas", "type_simplifie", "taux_manquants_%", "nb_manquants"]]
    .head(15)
    .to_string(index=False)
)

print("\nRépartition par type_simplifie :")
print(profile["type_simplifie"].value_counts().to_string())

print("\nTop 15 variables avec le plus de modalités/uniques :")
print(
    profile[["variable", "type_simplifie", "nb_modalites_ou_valeurs_uniques"]]
    .sort_values("nb_modalites_ou_valeurs_uniques", ascending=False)
    .head(15)
    .to_string(index=False)
)

# =============================================================================
# 4) EXPORTS (CSV + XLSX)
# =============================================================================
print("\n" + "=" * 90)
print("EXPORTS")
print("=" * 90)

# CSV
profile.to_csv(OUT_PROFILE_CSV, index=False, sep=";")
print("OK CSV :", OUT_PROFILE_CSV)

# Colonnes clés pour l’onglet "resume"
cols_show = [
    "variable",
    "dtype_pandas",
    "type_simplifie",
    "nb_modalites_ou_valeurs_uniques",
    "taux_manquants_%",
    "modalites_principales",
]

# Sécurise si jamais une colonne change
cols_resume = [c for c in cols_show if c in profile.columns] + ["min", "p50", "max"]

with pd.ExcelWriter(OUT_PROFILE_XLSX, engine="openpyxl") as writer:
    profile.to_excel(writer, sheet_name="profil_variables", index=False)
    profile[cols_resume].to_excel(writer, sheet_name="resume", index=False)

print("OK XLSX:", OUT_PROFILE_XLSX)

print("\n" + "=" * 90)
print("TERMINÉ")
print("=" * 90)


PROFILING BDD_FINAL_new.csv
[1] Lecture : /Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/BDD_FINAL_new.csv
OK: 494,483 lignes | 113 colonnes
Aperçu colonnes (20 premières): ['COMMUNE', 'ARM', 'DCFLT', 'DCLT', 'AGEREVQ', 'GS', 'DEROU', 'DIPL', 'EMPL', 'ILT', 'ILTUU', 'IMMI', 'INATC', 'INEEM', 'INPOM', 'INPSM', 'IPONDI', 'LPRM', 'METRODOM', 'MOCO']

[2] Construction du profil variables…
OK: profil construit
→ 113 variables profilées

RÉSUMÉS (console)

Top 15 variables avec le plus de manquants :
   variable dtype_pandas type_simplifie  taux_manquants_%  nb_manquants
        ARM      float64        numeric          100.0000        494483
      DEROU       object    categorical           99.3745        491390
      TYPMR      float64        numeric            0.6255          3093
       LPRM      float64        numeric            0.6255          3093
      NPERR      float64        numeric            0.6255          3093
       TYPL      float64        numeric            0.62

## 4. Stratégie de sélection des variables par apprentissage automatique

### 4.1. Objectif et positionnement méthodologique

L’objectif de cette étape n’est pas la prédiction du statut de travailleur transfrontalier, mais la **réduction rigoureuse de la dimension de l’espace des covariables** en vue de l’estimation d’un modèle probit interprétable.

L’apprentissage automatique est utilisé ici **exclusivement comme un outil de pré-sélection des variables explicatives**, et non comme un modèle structurel ou causal. Le modèle final retenu pour l’analyse économique reste un **probit binaire classique**, estimé sur un sous-ensemble de variables sélectionnées de manière systématique et reproductible.

Cette approche s’inscrit dans la littérature récente combinant méthodes économétriques traditionnelles et techniques de régularisation, tout en respectant les exigences d’interprétabilité et de transparence propres à l’analyse micro-économétrique.

---

### 4.2. Population d’étude et variable dépendante

La population étudiée est constituée des **actifs résidant dans la région Grand Est**, après appariement des bases INDCVI et MOBPRO. Le périmètre géographique est donc fixé ex ante, ce qui rend la variable REGION constante et exclue du jeu de covariables.

La variable dépendante est définie comme un indicateur binaire du statut de travailleur transfrontalier :

\[
Y_i = \mathbb{1}(\text{individu } i \text{ exerce son activité professionnelle à l’étranger})
\]

Ce statut est construit à partir des informations relatives au lieu de travail (codes DCFLT, DCLT, REGLT).

---

### 4.3. Exclusion a priori des variables non admissibles (anti-leakage)

Avant toute étape de sélection automatique, un filtrage strict des variables explicatives est opéré afin d’éviter toute fuite d’information (data leakage) ou toute identification artificielle du statut transfrontalier.

Sont exclues a priori du jeu de covariables :

- **Variables définissant directement la variable dépendante** :
  - DCFLT, DCLT, REGLT
  - ILT, ILTUU
- **Identifiants et quasi-identifiants** :
  - ID_INDIVIDU, NUMMI
  - COMMUNE, DEPCOM, CANTVILLE
  - IRIS et TRIRIS utilisés à un niveau trop fin
- **Variables constantes ou dégénérées** :
  - ARM (100 % manquante)
  - DEROU (quasi intégralement manquante)
  - METRODOM, RECH, REGION (constantes)
- **Variables postérieures ou mécaniquement liées à l’activité transfrontalière** :
  - Variables de localisation précise du lieu de travail
  - Variables de transport (TRANS), exclues du modèle principal et réservées à des analyses complémentaires

Ce filtrage garantit que l’ensemble des variables candidates est **prédéterminé** par rapport au statut transfrontalier et économiquement interprétable.

---

### 4.4. Structuration du jeu de covariables admissibles

Les variables restantes sont organisées en blocs analytiques cohérents afin de guider l’étape de sélection et de faciliter l’interprétation économique :

- **Démographie individuelle**  
  (âge, sexe, origine, statut migratoire)
- **Capital humain**  
  (niveau de diplôme, catégorie socioprofessionnelle)
- **Situation familiale et composition du ménage**  
  (couple, structure familiale, nombre d’enfants, taille du ménage)
- **Caractéristiques du logement et ancrage résidentiel**  
  (type de logement, statut d’occupation, surface, ancienneté, équipement)
- **Caractéristiques de l’emploi**  
  (type de contrat, temps de travail, secteur d’activité)
- **Contexte géographique agrégé**  
  (département de résidence, catégorie d’IRIS, niveau d’appariement)

Cette structuration permet de limiter les risques de sélection erratique et de faciliter les analyses de robustesse par blocs.

---

### 4.5. Prétraitement des données avant sélection

Avant l’estimation des modèles pénalisés, plusieurs traitements sont appliqués :

- Regroupement des modalités rares afin d’éviter une fragmentation excessive de l’espace des covariables.
- Traitement explicite des modalités de type *Z / ZZ* comme des modalités structurelles spécifiques.
- Encodage des variables catégorielles par indicatrices (one-hot encoding).
- Standardisation des variables numériques afin d’assurer la comparabilité des coefficients pénalisés.

Ces étapes sont intégrées dans un pipeline reproductible afin de garantir la stabilité des résultats.

---

### 4.6. Méthode de sélection par régression pénalisée

La sélection des variables est réalisée à l’aide d’une **régression logistique pénalisée**, choisie pour sa stabilité numérique et sa compatibilité avec les procédures de validation croisée.

Le modèle estimé est de la forme :

\[
\Pr(Y_i = 1 \mid X_i) = \Lambda(X_i'\beta)
\]

où \(\Lambda(\cdot)\) désigne la fonction logistique, et où le vecteur \(\beta\) est estimé sous contrainte de pénalisation.

Deux méthodes sont mobilisées :

- **Elastic Net** (pénalisation mixte L1–L2) comme approche principale, adaptée à la présence de covariables corrélées.
- **LASSO** (L1) comme test de robustesse.

Les hyperparamètres sont sélectionnés par validation croisée (k-fold).

---

### 4.7. Sélection par stabilité

Afin d’éviter une sélection dépendante d’un seul tirage de l’échantillon, une procédure de **stability selection** est mise en œuvre :

- Estimation du modèle pénalisé sur plusieurs sous-échantillons ou plis de validation croisée.
- Calcul de la fréquence de sélection de chaque variable.
- Conservation des variables sélectionnées dans une proportion minimale des réplications (seuil fixé ex ante).

Cette approche permet d’identifier un noyau de variables robustes et économiquement pertinentes.

---

### 4.8. Estimation du modèle probit final

Le sous-ensemble de variables sélectionnées est ensuite utilisé pour estimer un **modèle probit binaire standard** sur l’échantillon complet :

\[
\Pr(Y_i = 1 \mid X_i) = \Phi(X_i'\gamma)
\]

où \(\Phi(\cdot)\) désigne la fonction de répartition de la loi normale.

L’interprétation repose sur les **effets marginaux moyens**, estimés avec des erreurs-types robustes. L’accent est mis sur les ordres de grandeur, les signes et la cohérence économique des effets, plutôt que sur la significativité statistique brute post-sélection.

---

### 4.9. Analyses de robustesse

Plusieurs vérifications sont menées afin d’évaluer la robustesse des résultats :

- Comparaison des sélections Elastic Net et LASSO.
- Ré-estimation du probit avec et sans certains blocs de variables (logement, famille).
- Sensibilité à la pondération.
- Estimations sur sous-échantillons.

Ces tests permettent de s’assurer que les résultats ne reposent pas sur un choix arbitraire de spécification.

---

### 4.10. Positionnement interprétatif

Conformément aux bonnes pratiques, cette démarche ne vise pas à établir une relation causale stricte, mais à fournir une **analyse économétrique interprétable et robuste** des déterminants associés au statut de travailleur transfrontalier, en combinant rigueur économétrique et outils modernes de sélection de variables.


In [11]:
# ============================================================
# CODE 1 — DIAGNOSTIC OBLIGATOIRE (ultra minimal & propre)
# - Lit correctement le CSV (sep=';')
# - Vérifie uniquement la cible TRANSFRONTALIER (exact match)
# - Produit 3 fichiers : summary / flags / checks
# ============================================================

import os
import re
import numpy as np
import pandas as pd

# ----------------------------
# Paramètres
# ----------------------------
IN_PATH = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/BDD_FINAL_new.csv"
OUT_DIR = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/DIAGNOSTICS_ML"
TARGET = "TRANSFRONTALIER"
SEP = ";"

# Seuils
TH_MISS_HIGH = 0.95
TH_CONST_DOMINANT = 0.999
TH_HIGH_CARD_ABS = 5000
TH_HIGH_CARD_REL = 0.05
TH_EMPTY_STR_HIGH = 0.05

# Patterns “suspects” (leakage / ID / geo fine)
SUSPECT_PATTERNS = [
    r"\bwork\b", r"pays", r"dcflt", r"dclt", r"reglt",
    r"lux", r"suisse", r"allem", r"belg", r"etrang",
    r"iris", r"triris", r"depc", r"commune", r"cantville",
    r"\bid\b", r"nummi", r"ident"
]

# ----------------------------
# I/O checks + chargement
# ----------------------------
if not os.path.exists(IN_PATH):
    raise FileNotFoundError(f"Fichier introuvable: {IN_PATH}")

os.makedirs(OUT_DIR, exist_ok=True)

df = pd.read_csv(IN_PATH, sep=SEP, low_memory=False)

# Normalisation des noms (anti BOM / espaces)
df.columns = (
    pd.Index(df.columns)
      .astype(str)
      .str.replace("\ufeff", "", regex=False)
      .str.strip()
)

n, p = df.shape
print(f"[OK] Lecture: {n:,} lignes | {p:,} colonnes")

if p == 1:
    raise ValueError("Lecture en 1 seule colonne -> séparateur incorrect. Le fichier doit être en ';'.")

if TARGET not in df.columns:
    raise ValueError(f"Colonne cible '{TARGET}' absente. Colonnes dispo (extrait): {list(df.columns)[:30]} ...")

# ----------------------------
# Checks dataset globaux
# ----------------------------
checks_lines = []
checks_lines += [
    f"Source: {IN_PATH}",
    f"sep: {SEP}",
    f"Shape: {n} rows | {p} cols",
    f"Target: {TARGET}",
]

y = df[TARGET]
checks_lines.append(f"Target dtype: {y.dtype}")
checks_lines.append(f"Target missing: {y.isna().sum()} ({y.isna().mean():.4%})")

y_num = pd.to_numeric(y, errors="coerce")
uniq = set(y_num.dropna().unique().tolist())
checks_lines.append(f"Target unique values (numeric coercion): {sorted(list(uniq))}")

if uniq.issubset({0, 1}):
    checks_lines.append(f"Target proportion of 1s: {float(y_num.mean()):.4%}")
else:
    checks_lines.append("WARNING: Target is not strictly {0,1}.")

dup_rows = int(df.duplicated().sum())
checks_lines.append(f"Duplicate rows (exact duplicates): {dup_rows} ({dup_rows/n:.4%})")

# ----------------------------
# Profil colonne par colonne
# ----------------------------
rows = []
flags = []

high_card_rel_abs = max(int(np.ceil(TH_HIGH_CARD_REL * n)), TH_HIGH_CARD_ABS)

for col in df.columns:
    s = df[col]
    dtype = str(s.dtype)

    n_missing = int(s.isna().sum())
    miss_rate = float(n_missing / n)
    nunique = int(s.nunique(dropna=False))

    vc = s.value_counts(dropna=False)
    dom_count = int(vc.iloc[0]) if len(vc) else 0
    dom_share = float(dom_count / n) if n else np.nan
    dom_value = vc.index[0] if len(vc) else np.nan

    empty_share = np.nan
    if s.dtype == "object":
        empty_share = float((s.fillna("").astype(str).str.len() == 0).mean())

    inf_count = 0
    if pd.api.types.is_numeric_dtype(s):
        inf_count = int(np.isinf(s.to_numpy()).sum())

    col_l = col.lower()
    suspect_name = any(re.search(pat, col_l) for pat in SUSPECT_PATTERNS)

    rows.append({
        "variable": col,
        "dtype": dtype,
        "nunique_dropnaFalse": nunique,
        "missing_rate": miss_rate,
        "dominant_share": dom_share,
        "dominant_value": str(dom_value)[:120],
        "empty_string_share": empty_share,
        "inf_count": inf_count,
        "suspect_name_by_pattern": int(suspect_name),
        "is_target": int(col == TARGET),
    })

    reasons = []

    if col == TARGET:
        if miss_rate > 0:
            reasons.append("TARGET_HAS_MISSING")
        if not uniq.issubset({0, 1}):
            reasons.append("TARGET_NOT_BINARY_0_1")
    else:
        if miss_rate >= TH_MISS_HIGH:
            reasons.append(f"HIGH_MISSING(>={TH_MISS_HIGH:.0%})")
        if nunique <= 1:
            reasons.append("CONSTANT")
        elif dom_share >= TH_CONST_DOMINANT:
            reasons.append(f"QUASI_CONSTANT(dom>={TH_CONST_DOMINANT:.2%})")
        if nunique >= high_card_rel_abs:
            reasons.append(f"HIGH_CARDINALITY(>={high_card_rel_abs})")
        if not np.isnan(empty_share) and empty_share >= TH_EMPTY_STR_HIGH:
            reasons.append(f"MANY_EMPTY_STRINGS(>={TH_EMPTY_STR_HIGH:.0%})")
        if inf_count > 0:
            reasons.append("HAS_INF_VALUES")
        if suspect_name:
            reasons.append("SUSPECT_NAME_PATTERN")

    if reasons:
        flags.append({"variable": col, "reasons": ";".join(reasons)})

summary_df = pd.DataFrame(rows).sort_values(["missing_rate", "nunique_dropnaFalse"], ascending=[False, False])
flags_df = pd.DataFrame(flags).sort_values("variable")

# ----------------------------
# Exports
# ----------------------------
summary_path = os.path.join(OUT_DIR, "DIAG_stepA_summary.csv")
flags_path = os.path.join(OUT_DIR, "DIAG_stepA_flags.csv")
checks_path = os.path.join(OUT_DIR, "DIAG_stepA_dataset_checks.txt")

summary_df.to_csv(summary_path, index=False)
flags_df.to_csv(flags_path, index=False)

with open(checks_path, "w", encoding="utf-8") as f:
    f.write("\n".join(checks_lines))

print(f"\n[OK] Export summary → {summary_path}")
print(f"[OK] Export flags   → {flags_path}")
print(f"[OK] Export checks  → {checks_path}")

print("\n=== QUICK PEEK: Top 15 missing_rate ===")
print(summary_df[["variable","dtype","missing_rate","nunique_dropnaFalse","dominant_share","suspect_name_by_pattern"]].head(15))

print("\n=== QUICK PEEK: Flags (first 40) ===")
print(flags_df.head(40))


[OK] Lecture: 494,483 lignes | 113 colonnes

[OK] Export summary → /Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/DIAGNOSTICS_ML/DIAG_stepA_summary.csv
[OK] Export flags   → /Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/DIAGNOSTICS_ML/DIAG_stepA_flags.csv
[OK] Export checks  → /Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/DIAGNOSTICS_ML/DIAG_stepA_dataset_checks.txt

=== QUICK PEEK: Top 15 missing_rate ===
        variable    dtype  missing_rate  nunique_dropnaFalse  dominant_share  \
1            ARM  float64      1.000000                    1        1.000000   
6          DEROU   object      0.993745                    2        0.993745   
30         TYPMR  float64      0.006255                   10        0.548737   
17          LPRM  float64      0.006255                    8        0.631555   
21         NPERR  float64      0.006255                    7        0.294358   
29          TYPL  float64      0.006255                    7        0

## Conclusion du diagnostic préalable (Code 1) et implications pour la préparation ML

Le Code 1 a permis de réaliser un diagnostic exhaustif de la base de données utilisée pour l’analyse des déterminants du travail transfrontalier. Cette étape est indispensable afin de garantir la qualité statistique, la cohérence économétrique et la validité méthodologique des modèles estimés ultérieurement.

### 1. Validation globale de la base de données

La base finale comprend **494 483 observations et 113 variables**, ce qui constitue un échantillon de grande taille, adapté à des méthodes de sélection de variables par apprentissage automatique et à l’estimation de modèles non linéaires (probit).

La variable cible **`TRANSFRONTALIER`** :
- est bien présente dans la base,
- ne présente pas de valeurs manquantes,
- est strictement binaire (0/1),
- présente une distribution compatible avec une modélisation probabiliste.

Aucune incohérence majeure (duplicats massifs, erreurs de lecture, corruption de colonnes) n’a été détectée au niveau global du jeu de données.

---

### 2. Identification des variables non exploitables en l’état

Le diagnostic met en évidence plusieurs catégories de variables problématiques pour une utilisation directe dans un cadre ML / économétrique.

#### 2.1 Variables à taux de valeurs manquantes extrême ou constantes

Certaines variables présentent :
- un taux de valeurs manquantes supérieur à 95 %,
- ou une variance nulle (variables constantes).

Exemples :
- `ARM`
- `DEROU`
- `METRODOM`
- `RECH`
- `REGION`
- `TACT`
- `is_unique_key`

Ces variables **n’apportent aucune information exploitable** et seront **exclues définitivement** de la base ML.

---

#### 2.2 Variables à très forte cardinalité (identifiants techniques)

Certaines variables présentent une cardinalité quasi unique, les rendant impropres à toute modélisation explicative ou prédictive standard :

- `ID_INDIVIDU`
- `NUMMI`

Ces variables sont assimilables à des **identifiants techniques** et seront **exclues** de la base ML afin d’éviter toute sur-adaptation ou fuite d’information.

---

### 3. Variables géographiques fines et variables potentiellement fuyantes

Le diagnostic a identifié plusieurs variables marquées comme *suspectes* au sens méthodologique, car elles décrivent soit :
- des localisations géographiques très fines,
- soit des informations directement liées au lieu de travail à l’étranger.

Cela concerne notamment :
- `COMMUNE`, `DEPCOM`, `CANTVILLE`, `IRIS`, `TRIRIS`, `CATIRIS`
- `DCLT`, `DCFLT`, `REGLT`
- les variables `WORK_*` (`WORK_ALLEMAGNE`, `WORK_SUISSE`, `WORK_LUXEMBOURG`, `WORK_BELGIQUE`, `WORK_IN_PAYS`)

Ces variables ne sont **pas erronées**, mais leur utilisation nécessite un **choix économétrique explicite**, car elles sont fortement corrélées à la variable cible.

---

### 4. Positionnement méthodologique retenu pour la suite

Conformément à l’objectif de l’étude, une distinction claire est opérée entre :

- **Étape de sélection des variables (ML)**  
- **Étape d’estimation du modèle final (probit)**

#### 4.1 Pour la sélection ML (Code 2)

Le ML est utilisé **uniquement comme outil de sélection de variables pertinentes**, et non comme modèle final.

À ce stade :
- les variables constantes, quasi constantes ou à très forte cardinalité sont exclues,
- les identifiants techniques sont exclus,
- les variables explicitement post-déterminées par le statut transfrontalier (notamment `WORK_*`) sont **exclues du processus de sélection ML**, afin d’éviter toute domination mécanique de la sélection et toute fuite d’information.

Le ML opère donc sur un ensemble de variables **ex ante**, décrivant les caractéristiques individuelles, familiales, professionnelles et résidentielles des individus.

---

#### 4.2 Pour le modèle probit final

Le modèle probit a un objectif **prédictif**, mais également **analytique**. À ce titre :

- les variables sélectionnées par le ML constituent le socle du modèle principal,
- les variables `WORK_*`, qui capturent des effets structurels liés au pays de travail (attractivité économique, fiscale, institutionnelle), pourront être **réintroduites de manière contrôlée** dans le modèle probit ou mobilisées dans des extensions (analyses conditionnelles, modèles par pays, modèles multinomiaux).

Ce choix permet de préserver :
- la cohérence économétrique,
- l’interprétabilité des coefficients,
- et la robustesse des résultats.

---

### 5. Implications pour le Code 2 – Préparation de la base ML

Le Code 2 consistera donc à :

1. Construire une base ML nettoyée :
   - exclusion des variables non informatives,
   - exclusion des identifiants et variables constantes,
   - séparation explicite des variables réservées aux extensions.

2. Encoder correctement les variables catégorielles retenues.

3. Préparer une matrice de données strictement adaptée à la sélection de variables par méthodes pénalisées (LASSO / Elastic Net).

Cette préparation repose directement sur les résultats du diagnostic du Code 1 et garantit la solidité méthodologique de l’ensemble de l’analyse.


In [16]:
# ============================================================
# CODE 2 — PRÉPARATION DE LA BDD ML (VERSION CORRIGÉE ET COHÉRENTE)
# Objectif : créer une base "prête ML" UNIQUEMENT pour la
# SÉLECTION DE VARIABLES (Elastic Net / Lasso)
#
# PRINCIPES STRICTS :
# 1) Exclusion définitive :
#    - variables constantes / quasi-nulles
#    - identifiants
#    - variables d’ingénierie / appariement
#
# 2) Exclusion TEMPORAIRE (phase ML uniquement) :
#    - variables de leakage (lieu/pays de travail)
#
# 3) Exclusion définitive :
#    - géographie fine ou proxy géographique
#
# 4) AUCUN choix économique ici :
#    - les arbitrages se feront PLUS TARD via Elastic Net
#
# Sorties :
#  - X_ml_ready.csv
#  - y.csv
#  - dropped_columns.csv
#  - kept_raw_features.csv
# ============================================================

import os
import json
import numpy as np
import pandas as pd

# ----------------------------
# 0) Paramètres
# ----------------------------
IN_PATH = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/BDD_FINAL_new.csv"
OUT_DIR = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/ML_READY"

SEP = ";"
TARGET = "TRANSFRONTALIER"
WEIGHT = "IPONDI"   # conservé à part, jamais feature ML

TOP_K_LEVELS = 50
MIN_FREQ = 200

# ----------------------------
# 1) Chargement
# ----------------------------
os.makedirs(OUT_DIR, exist_ok=True)

df = pd.read_csv(IN_PATH, sep=SEP, low_memory=False)
df.columns = (
    df.columns.astype(str)
    .str.replace("\ufeff", "", regex=False)
    .str.strip()
)

n, p = df.shape
print(f"[OK] Lecture : {n:,} lignes | {p:,} colonnes")

if TARGET not in df.columns:
    raise ValueError(f"Colonne cible '{TARGET}' absente.")

# ----------------------------
# 2) LISTES D’EXCLUSION — VALIDÉES MÉTHODOLOGIQUEMENT
# ----------------------------

# A. Variables NON INFORMATIVES / TECHNIQUES (EXCLUSION DÉFINITIVE)
DROP_ALWAYS = [
    "ARM", "DEROU", "METRODOM", "RECH", "REGION", "TACT", "is_unique_key"
]

# B. Identifiants
DROP_IDENTIFIERS = [
    "ID_INDIVIDU", "NUMMI"
]

# C. Variables d’INGÉNIERIE / APPARIEMENT (ERREUR MAJEURE SI CONSERVÉES)
DROP_ENGINEERING = [
    "COMMUNE", "DEPCOM", "IRIS", "TRIRIS",
    "CANTVILLE", "CATIRIS", "NIVEAU_APPARIEMENT"
]

# D. Variables de LEAKAGE (exclues POUR LA PHASE ML SEULEMENT)
DROP_LEAKAGE_FOR_SELECTION = [
    "WORK_IN_PAYS", "WORK_LUXEMBOURG", "WORK_ALLEMAGNE",
    "WORK_SUISSE", "WORK_BELGIQUE",
    "DCLT", "DCFLT", "REGLT"
]

# E. Poids (jamais feature ML)
DROP_WEIGHT = [WEIGHT] if WEIGHT in df.columns else []

# Ensemble final des exclusions
DROP_ALL = set(
    DROP_ALWAYS
    + DROP_IDENTIFIERS
    + DROP_ENGINEERING
    + DROP_LEAKAGE_FOR_SELECTION
    + DROP_WEIGHT
)

# ----------------------------
# 3) Séparation cible / features
# ----------------------------
y = df[TARGET].copy()

X_raw = df.drop(
    columns=[c for c in DROP_ALL if c in df.columns] + [TARGET],
    errors="ignore"
)

print(f"[OK] Features candidates conservées : {X_raw.shape[1]}")

# Traçabilité
dropped_effective = sorted([c for c in df.columns if c not in X_raw.columns and c != TARGET])

# ----------------------------
# 4) Préparation des variables
# ----------------------------
def collapse_rare_levels(s, top_k=TOP_K_LEVELS, min_freq=MIN_FREQ):
    s = s.astype("object").where(~s.isna(), "__MISSING__")
    vc = s.value_counts()
    keep = set(vc.head(top_k).index)
    keep |= set(vc[vc >= min_freq].index)
    return s.where(s.isin(keep), "__OTHER__")

# Séparation types
cat_cols = [c for c in X_raw.columns if X_raw[c].dtype == "object"]
num_cols = [c for c in X_raw.columns if c not in cat_cols]

# A. Catégorielles
X_cat = pd.DataFrame(index=X_raw.index)
cat_info = []

for c in cat_cols:
    s = collapse_rare_levels(X_raw[c])
    X_cat[c] = s
    cat_info.append({
        "variable": c,
        "n_modalites_post": int(s.nunique())
    })

# B. Numériques (imputation médiane)
X_num = pd.DataFrame(index=X_raw.index)
num_info = []

for c in num_cols:
    s = pd.to_numeric(X_raw[c], errors="coerce")
    med = float(s.median())
    X_num[c] = s.fillna(med)
    num_info.append({
        "variable": c,
        "imputation": "median",
        "median": med
    })

# ----------------------------
# 5) One-Hot Encoding
# ----------------------------
X_cat_dum = pd.get_dummies(X_cat, drop_first=False)
X = pd.concat([X_num, X_cat_dum], axis=1)

# Sécurité finale
if X.isna().sum().sum() > 0:
    raise ValueError("Il reste des NaN dans X après préparation.")

print(f"[OK] Matrice ML finale : {X.shape[0]:,} x {X.shape[1]:,}")

# ----------------------------
# 6) Exports
# ----------------------------
X.to_csv(os.path.join(OUT_DIR, "X_ml_ready.csv"), index=False)
y.to_csv(os.path.join(OUT_DIR, "y.csv"), index=False)

pd.DataFrame({"variable": dropped_effective}).to_csv(
    os.path.join(OUT_DIR, "dropped_columns.csv"), index=False
)

pd.DataFrame({"variable": X_raw.columns}).to_csv(
    os.path.join(OUT_DIR, "kept_raw_features.csv"), index=False
)

with open(os.path.join(OUT_DIR, "categorical_info.json"), "w") as f:
    json.dump(cat_info, f, indent=2, ensure_ascii=False)

with open(os.path.join(OUT_DIR, "numeric_info.json"), "w") as f:
    json.dump(num_info, f, indent=2, ensure_ascii=False)

print("[OK] Exports terminés – base prête pour Elastic Net")


[OK] Lecture : 494,483 lignes | 113 colonnes
[OK] Features candidates conservées : 87
[OK] Matrice ML finale : 494,483 x 452
[OK] Exports terminés – base prête pour Elastic Net


# Arbitrages conceptuels des variables explicatives

Dans la base issue de l’appariement des fichiers INDCVI et MOBPRO, plusieurs dimensions
socio-économiques sont décrites par des ensembles de variables alternatives ou
hiérarchiquement liées.  
Afin d’éviter toute redondance conceptuelle, instabilité statistique et perte
d’interprétabilité, des arbitrages économiquement motivés ont été opérés **en amont**
de toute procédure de sélection automatique par machine learning.

Le principe général retenu est le suivant :

> Pour une même dimension conceptuelle, nous conservons les variables apportant  
> (i) la plus forte granularité informationnelle utile,  
> (ii) une interprétabilité économétrique claire,  
> (iii) une compatibilité avec un modèle probit explicatif.

Les arbitrages finaux sont détaillés ci-dessous.

---

## 1. Âge

Variables candidates :
- AGED
- AGEREV
- AGEREVQ
- AGER20
- ANAI

Choix retenu :
- **AGED**

Justification :
L’âge exact constitue une variable continue permettant d’exploiter l’ensemble
de l’information disponible. Dans un cadre probit, il autorise l’introduction
éventuelle de non-linéarités (termes quadratiques ou splines) sans perte de précision,
contrairement aux classes d’âge qui introduisent des effets de seuil arbitraires.

---

## 2. Statut d’activité et position socio-professionnelle

Variables candidates :
- STAT, STATR
- EMPL
- PCSL
- GS, STAT_GSEC

Choix retenus :
- **STATR** (statut d’activité détaillé)
- **PCSL** (position socio-professionnelle)
- **GS** (statut public / privé)

Justification :
Le statut d’activité, la position socio-professionnelle et le statut institutionnel
capturent des dimensions distinctes du marché du travail. Les conserver conjointement
permet de différencier l’effet de l’insertion sur le marché du travail, de la
stratification sociale et du cadre institutionnel de l’emploi.

---

## 3. Mobilité domicile–travail

Variables candidates :
- ILT
- ILTUU
- TRANS

Choix retenus :
- **ILT**
- **TRANS**

Justification :
ILT mesure l’intensité ou la distance des déplacements domicile–travail et constitue
une variable quantitative particulièrement informative. TRANS capte le mode de
transport, apportant une information complémentaire sur les contraintes et choix
de mobilité. ILTUU est dominée par ILT et n’est pas conservée.

---

## 4. Taille et structure du ménage

Variables candidates :
- NPERR
- INPER, INPERF
- INFAM
- INPSM
- INEEM

Choix retenus :
- **NPERR**
- **INFAM**

Justification :
NPERR fournit la mesure la plus granulaire de la taille du ménage. INFAM permet de
distinguer la structure familiale (famille / non-famille), dimension complémentaire
non redondante avec la taille.

---

## 5. Présence d’enfants

Variables candidates :
- NENFR
- NE3FR
- NE5FR
- NE17FR
- NE24FR

Choix retenu :
- **NE17FR**

Justification :
NE17FR identifie la présence et le nombre d’enfants à charge dans une tranche d’âge
économiquement pertinente, directement liée aux contraintes de mobilité et aux
arbitrages professionnels.

---

## 6. Typologie du ménage

Variables candidates :
- MOCO
- MOCO_DET
- TYPMC
- TYPMR
- TDM8
- SFM

Choix retenus :
- **TDM8**
- **SFM**

Justification :
TDM8 offre une typologie synthétique et robuste des ménages, limitant la fragmentation
excessive des modalités. SFM complète cette information en distinguant les formes
familiales essentielles, sans introduire de bruit catégoriel inutile.

---

## 7. Conditions de logement

Variables candidates :
- SURF
- HLML
- GARL
- BAIN, WC, CHAU, CLIM, CUIS, EAU, EGOUL, ELEC
- BATI, CATL, CATPC

Choix retenus :
- **SURF**
- **HLML**
- **CATPC**

Justification :
SURF capte la surface du logement et constitue un proxy robuste du niveau de vie.
HLML distingue le logement social, fortement corrélé à la position socio-économique.
CATPC synthétise le type de logement de manière plus informative que les indicateurs
binaires très faiblement discriminants.

---

## 8. Origine et migration

Variables candidates :
- IMMI
- INATC
- IRAN
- DNAI
- ORIDT

Choix retenus :
- **IMMI**
- **INATC**

Justification :
IMMI identifie le statut d’immigré, tandis que INATC renseigne la nationalité actuelle.
Les autres variables sont trop fines ou indirectes et n’apportent pas d’information
supplémentaire robuste pour l’analyse.

---

## 9. Formation et éducation

Variables candidates :
- DIPL
- ETUD
- ILETUD

Choix retenus :
- **DIPL**
- **ETUD**

Justification :
DIPL mesure le niveau de diplôme atteint, variable clé pour l’accès au marché du
travail transfrontalier. ETUD permet d’identifier les individus encore en formation.
ILETUD est redondante et n’est pas conservée.

---

## 10. Secteur d’activité

Variables candidates :
- NA5
- NA17

Choix retenu :
- **NA17**

Justification :
NA17 fournit une désagrégation sectorielle plus fine, permettant de mieux capter
les effets sectoriels spécifiques sur la probabilité de travail transfrontalier.

---

## 11. Santé

Variables candidates :
- SANI
- SANIDOM

Choix retenu :
- **SANI**

Justification :
SANI décrit l’état de santé individuel, susceptible d’influencer directement la
mobilité professionnelle. SANIDOM est liée au logement et déjà couverte par d’autres
variables retenues.

---

## Conclusion

Les arbitrages opérés permettent de constituer une base explicative cohérente,
économiquement fondée et interprétable.  
La sélection automatique par machine learning (Elastic Net) intervient uniquement
en aval, pour gérer la colinéarité résiduelle et stabiliser l’estimation, sans se
substituer aux choix conceptuels de l’économiste.


In [18]:
# ============================================================
# CODE 3 — CONSTRUCTION DE LA BDD "ARBITRAGES ÉCONOMIQUES" (post CODE 2)
# Version améliorée avec affichage détaillé dans la console
# ============================================================

import os
import pandas as pd
from datetime import datetime

# ----------------------------
# 0) Paramètres
# ----------------------------
ML_READY_DIR = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/ML_READY"
X_PATH = os.path.join(ML_READY_DIR, "X_ml_ready.csv")
Y_PATH = os.path.join(ML_READY_DIR, "y.csv")

OUT_DIR = os.path.join(ML_READY_DIR, "FINAL_AFTER_ARBITRAGES")
os.makedirs(OUT_DIR, exist_ok=True)

OUT_X = os.path.join(OUT_DIR, "X_ml_final_arbitrages.csv")
OUT_Y = os.path.join(OUT_DIR, "y_final.csv")
OUT_REPORT = os.path.join(OUT_DIR, "keep_features_report.txt")

# ----------------------------
# Fonctions utilitaires d'affichage
# ----------------------------
def print_header(title):
    """Affiche un en-tête formaté"""
    width = 70
    print("\n" + "=" * width)
    print(f" {title}".center(width))
    print("=" * width)

def print_section(title):
    """Affiche un titre de section"""
    print(f"\n{'─' * 50}")
    print(f"  {title}")
    print(f"{'─' * 50}")

def print_var_detail(var_name, cols_found, all_cols):
    """Affiche le détail d'une variable conceptuelle"""
    if len(cols_found) == 0:
        print(f"  ❌ {var_name:<15} → NON TROUVÉE")
    elif len(cols_found) == 1 and cols_found[0] == var_name:
        # Variable numérique (pas de dummies)
        print(f"  ✓ {var_name:<15} → numérique (1 colonne)")
    else:
        # Variable catégorielle avec dummies
        print(f"  ✓ {var_name:<15} → catégorielle ({len(cols_found)} dummies)")

# ----------------------------
# 1) Liste des variables conceptuelles retenues
# ----------------------------
FINAL_CONCEPTUAL_VARS = [
    # 1) Âge
    "AGED",
    # 2) Statut d'activité / PCS / Public-privé
    "STATR", "PCSL", "GS",
    # 3) Mobilité domicile–travail
    "ILT", "TRANS",
    # 4) Taille/structure ménage
    "NPERR", "INFAM",
    # 5) Présence d'enfants
    "NE17FR",
    # 6) Typologie ménage
    "TDM8", "SFM",
    # 7) Logement
    "SURF", "HLML", "CATPC",
    # 8) Origine / migration
    "IMMI", "INATC",
    # 9) Formation / éducation
    "DIPL", "ETUD",
    # 10) Secteur d'activité
    "NA17",
    # 11) Santé
    "SANI",
]

# Regroupement thématique pour l'affichage
VAR_GROUPS = {
    "Âge": ["AGED"],
    "Statut d'activité / PCS": ["STATR", "PCSL", "GS"],
    "Mobilité domicile-travail": ["ILT", "TRANS"],
    "Taille/structure ménage": ["NPERR", "INFAM"],
    "Présence d'enfants": ["NE17FR"],
    "Typologie ménage": ["TDM8", "SFM"],
    "Logement": ["SURF", "HLML", "CATPC"],
    "Origine / migration": ["IMMI", "INATC"],
    "Formation / éducation": ["DIPL", "ETUD"],
    "Secteur d'activité": ["NA17"],
    "Santé": ["SANI"],
}

# ----------------------------
# 2) Démarrage et chargement
# ----------------------------
print_header("CODE 3 — FILTRAGE POST-ARBITRAGES ÉCONOMIQUES")
print(f"\n📅 Exécution : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print_section("1. CHARGEMENT DES DONNÉES")

if not os.path.exists(X_PATH):
    raise FileNotFoundError(f"X introuvable : {X_PATH}")
if not os.path.exists(Y_PATH):
    raise FileNotFoundError(f"y introuvable : {Y_PATH}")

print(f"  📂 Répertoire source : {ML_READY_DIR}")
print(f"  📄 Fichier X : X_ml_ready.csv")
print(f"  📄 Fichier y : y.csv")
print(f"\n  ⏳ Chargement en cours...")

X = pd.read_csv(X_PATH, low_memory=False)
y = pd.read_csv(Y_PATH, low_memory=False)

print(f"\n  ✅ X chargé avec succès")
print(f"     • Observations : {X.shape[0]:,}")
print(f"     • Colonnes (features) : {X.shape[1]:,}")
print(f"     • Taille mémoire : {X.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

print(f"\n  ✅ y chargé avec succès")
print(f"     • Observations : {y.shape[0]:,}")
print(f"     • Colonnes : {y.shape[1]}")

# Distribution de y
if y.shape[1] == 1:
    y_col = y.iloc[:, 0]
    print(f"\n  📊 Distribution de la variable cible :")
    value_counts = y_col.value_counts().sort_index()
    total = len(y_col)
    for val, count in value_counts.items():
        pct = count / total * 100
        print(f"     • {val} : {count:,} ({pct:.2f}%)")

# ----------------------------
# 3) Sélection des colonnes
# ----------------------------
print_section("2. MAPPING VARIABLES CONCEPTUELLES → COLONNES")

print(f"\n  Variables conceptuelles demandées : {len(FINAL_CONCEPTUAL_VARS)}")
print(f"  Colonnes disponibles dans X : {X.shape[1]:,}")

cols = list(X.columns)
keep_cols = []
not_found = []
var_mapping = {}  # Pour stocker le détail du mapping

for v in FINAL_CONCEPTUAL_VARS:
    matched = []
    
    # Cas numérique ou déjà encodé en une seule colonne
    if v in cols:
        matched.append(v)
    
    # Cas catégoriel one-hot
    prefix = v + "_"
    matched += [c for c in cols if c.startswith(prefix)]
    
    # Dédoublonnage
    matched = sorted(set(matched))
    var_mapping[v] = matched
    
    if len(matched) == 0:
        not_found.append(v)
    else:
        keep_cols.extend(matched)

# Unique + ordre stable
keep_cols = list(dict.fromkeys(keep_cols))

if len(keep_cols) == 0:
    raise ValueError("Aucune colonne matchée. Vérifie les noms et le format des dummies.")

# Affichage détaillé par groupe thématique
print_section("3. DÉTAIL PAR GROUPE THÉMATIQUE")

total_numeric = 0
total_categorical = 0
total_dummies = 0

for group_name, group_vars in VAR_GROUPS.items():
    print(f"\n  📁 {group_name}")
    group_cols = 0
    
    for v in group_vars:
        matched = var_mapping.get(v, [])
        if len(matched) == 0:
            print(f"     ❌ {v:<12} → NON TROUVÉE")
        elif len(matched) == 1 and matched[0] == v:
            print(f"     ✓ {v:<12} → numérique (1 col)")
            total_numeric += 1
            group_cols += 1
        else:
            print(f"     ✓ {v:<12} → catégorielle ({len(matched)} dummies)")
            total_categorical += 1
            total_dummies += len(matched)
            group_cols += len(matched)
    
    print(f"     {'─' * 30}")
    print(f"     Sous-total groupe : {group_cols} colonnes")

# ----------------------------
# 4) Statistiques globales
# ----------------------------
print_section("4. STATISTIQUES GLOBALES DE SÉLECTION")

print(f"\n  📊 Résumé du mapping :")
print(f"     • Variables conceptuelles trouvées : {len(FINAL_CONCEPTUAL_VARS) - len(not_found)}/{len(FINAL_CONCEPTUAL_VARS)}")
print(f"     • Variables conceptuelles manquantes : {len(not_found)}")
if not_found:
    print(f"       → {', '.join(not_found)}")

print(f"\n  📐 Dimensions :")
print(f"     • Variables numériques : {total_numeric}")
print(f"     • Variables catégorielles : {total_categorical}")
print(f"     • Total dummies catégorielles : {total_dummies}")
print(f"     • Total colonnes sélectionnées : {len(keep_cols)}")

print(f"\n  📉 Réduction de dimension :")
print(f"     • Colonnes initiales : {X.shape[1]:,}")
print(f"     • Colonnes conservées : {len(keep_cols)}")
print(f"     • Colonnes supprimées : {X.shape[1] - len(keep_cols):,}")
print(f"     • Taux de réduction : {(1 - len(keep_cols)/X.shape[1])*100:.1f}%")

# ----------------------------
# 5) Construction de X_final
# ----------------------------
print_section("5. CONSTRUCTION DE LA BASE FINALE")

X_final = X[keep_cols].copy()

print(f"\n  📦 X_final créé :")
print(f"     • Shape : {X_final.shape[0]:,} × {X_final.shape[1]}")
print(f"     • Taille mémoire : {X_final.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Vérification des valeurs manquantes
missing_counts = X_final.isnull().sum()
cols_with_missing = missing_counts[missing_counts > 0]

print(f"\n  🔍 Vérification des valeurs manquantes :")
if len(cols_with_missing) == 0:
    print(f"     ✅ Aucune valeur manquante détectée")
else:
    print(f"     ⚠️  {len(cols_with_missing)} colonnes avec valeurs manquantes :")
    for col, count in cols_with_missing.head(10).items():
        pct = count / len(X_final) * 100
        print(f"        • {col}: {count:,} ({pct:.2f}%)")
    if len(cols_with_missing) > 10:
        print(f"        ... et {len(cols_with_missing) - 10} autres colonnes")

# Aperçu des types de données
print(f"\n  📋 Types de données :")
dtype_counts = X_final.dtypes.value_counts()
for dtype, count in dtype_counts.items():
    print(f"     • {dtype}: {count} colonnes")

# ----------------------------
# 6) Exports
# ----------------------------
print_section("6. EXPORT DES FICHIERS")

print(f"\n  ⏳ Export en cours...")

X_final.to_csv(OUT_X, index=False)
y.to_csv(OUT_Y, index=False)

# Calcul des tailles de fichiers
x_size = os.path.getsize(OUT_X) / 1024**2
y_size = os.path.getsize(OUT_Y) / 1024**2

print(f"\n  ✅ Fichiers exportés avec succès :")
print(f"     📄 X_final : {OUT_X}")
print(f"        → Taille : {x_size:.1f} MB")
print(f"     📄 y_final : {OUT_Y}")
print(f"        → Taille : {y_size:.1f} MB")

# ----------------------------
# 7) Génération du rapport
# ----------------------------
lines = []
lines.append("=" * 70)
lines.append("RAPPORT DE FILTRAGE POST-ARBITRAGES")
lines.append(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
lines.append("=" * 70)
lines.append("")
lines.append("SOURCES")
lines.append("-" * 40)
lines.append(f"X source : {X_PATH}")
lines.append(f"y source : {Y_PATH}")
lines.append("")
lines.append("STATISTIQUES")
lines.append("-" * 40)
lines.append(f"Variables conceptuelles demandées : {len(FINAL_CONCEPTUAL_VARS)}")
lines.append(f"Variables conceptuelles trouvées : {len(FINAL_CONCEPTUAL_VARS) - len(not_found)}")
lines.append(f"Variables conceptuelles manquantes : {len(not_found)}")
if not_found:
    lines.append(f"  → {', '.join(not_found)}")
lines.append("")
lines.append(f"Colonnes initiales : {X.shape[1]}")
lines.append(f"Colonnes conservées : {len(keep_cols)}")
lines.append(f"Taux de réduction : {(1 - len(keep_cols)/X.shape[1])*100:.1f}%")
lines.append("")
lines.append(f"Shape X_final : {X_final.shape[0]} × {X_final.shape[1]}")
lines.append("")
lines.append("DÉTAIL PAR GROUPE THÉMATIQUE")
lines.append("-" * 40)

for group_name, group_vars in VAR_GROUPS.items():
    lines.append(f"\n[{group_name}]")
    for v in group_vars:
        matched = var_mapping.get(v, [])
        if len(matched) == 0:
            lines.append(f"  ❌ {v} : NON TROUVÉE")
        elif len(matched) == 1 and matched[0] == v:
            lines.append(f"  ✓ {v} : numérique (1 colonne)")
        else:
            lines.append(f"  ✓ {v} : catégorielle ({len(matched)} dummies)")
            for col in matched[:5]:
                lines.append(f"      - {col}")
            if len(matched) > 5:
                lines.append(f"      ... et {len(matched) - 5} autres")

lines.append("")
lines.append("LISTE COMPLÈTE DES COLONNES CONSERVÉES")
lines.append("-" * 40)
for i, col in enumerate(keep_cols, 1):
    lines.append(f"  {i:3}. {col}")

lines.append("")
lines.append("FICHIERS EXPORTÉS")
lines.append("-" * 40)
lines.append(f"X_final : {OUT_X}")
lines.append(f"y_final : {OUT_Y}")

with open(OUT_REPORT, "w", encoding="utf-8") as f:
    f.write("\n".join(lines))

print(f"     📄 Rapport : {OUT_REPORT}")

# ----------------------------
# 8) Résumé final
# ----------------------------
print_header("RÉSUMÉ FINAL")

print(f"""
  ┌─────────────────────────────────────────────────────────┐
  │  FILTRAGE POST-ARBITRAGES TERMINÉ AVEC SUCCÈS           │
  ├─────────────────────────────────────────────────────────┤
  │  Observations     : {X_final.shape[0]:>10,}                       │
  │  Features initiales: {X.shape[1]:>10,}                       │
  │  Features finales  : {X_final.shape[1]:>10}                       │
  │  Réduction         : {(1 - len(keep_cols)/X.shape[1])*100:>10.1f}%                      │
  ├─────────────────────────────────────────────────────────┤
  │  Variables manquantes : {len(not_found):<5}                            │
  └─────────────────────────────────────────────────────────┘
""")

# Liste des 10 premières colonnes conservées
print("  📋 Aperçu des colonnes conservées (10 premières) :")
for i, col in enumerate(keep_cols[:10], 1):
    print(f"     {i:2}. {col}")
if len(keep_cols) > 10:
    print(f"     ... et {len(keep_cols) - 10} autres colonnes")

print(f"\n  📁 Fichiers disponibles dans :")
print(f"     {OUT_DIR}")
print()


             CODE 3 — FILTRAGE POST-ARBITRAGES ÉCONOMIQUES            

📅 Exécution : 2026-01-06 18:40:42

──────────────────────────────────────────────────
  1. CHARGEMENT DES DONNÉES
──────────────────────────────────────────────────
  📂 Répertoire source : /Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/ML_READY
  📄 Fichier X : X_ml_ready.csv
  📄 Fichier y : y.csv

  ⏳ Chargement en cours...

  ✅ X chargé avec succès
     • Observations : 494,483
     • Colonnes (features) : 452
     • Taille mémoire : 332.0 MB

  ✅ y chargé avec succès
     • Observations : 494,483
     • Colonnes : 1

  📊 Distribution de la variable cible :
     • 0 : 450,219 (91.05%)
     • 1 : 44,264 (8.95%)

──────────────────────────────────────────────────
  2. MAPPING VARIABLES CONCEPTUELLES → COLONNES
──────────────────────────────────────────────────

  Variables conceptuelles demandées : 20
  Colonnes disponibles dans X : 452

──────────────────────────────────────────────────
  3. DÉTAIL PA

In [ ]:
# ============================================================
# CONSTRUCTION DE LA BDD ML FINALE (avant Elastic Net)
# Basée exclusivement sur les arbitrages économiques validés
# ============================================================

import os
import pandas as pd
import numpy as np

# ----------------------------
# Paramètres
# ----------------------------
IN_PATH = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/BDD_FINAL_new.csv"
SEP = ";"

OUT_DIR = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/ML_FINAL"
os.makedirs(OUT_DIR, exist_ok=True)

TARGET = "TRANSFRONTALIER"
WEIGHT = "IPONDI"

# ----------------------------
# Variables retenues
# ----------------------------

NUM_VARS = [
    "AGED", "NPERR", "ILT", "DIPL", "GS", "EMPL"
]

CAT_VARS = [
    "STATR", "STAT_GSEC", "TRANS", "TYPL", "TYPMR",
    "SFM", "TDM8", "HLML", "CATPC", "INFAM",
    "NE17FR", "IMMI", "INATC", "ETUD",
    "NA17", "SANI", "SEXE"
]

FINAL_VARS = NUM_VARS + CAT_VARS

# ----------------------------
# Chargement (colonnes utiles seulement → rapide)
# ----------------------------
USECOLS = FINAL_VARS + [TARGET, WEIGHT]

df = pd.read_csv(
    IN_PATH,
    sep=SEP,
    usecols=USECOLS,
    low_memory=False
)

print(f"[OK] Lecture : {df.shape[0]:,} lignes | {df.shape[1]} colonnes")

# ----------------------------
# Séparation X / y / w
# ----------------------------
y = df[TARGET].astype(int)
w = df[WEIGHT]

X = df[FINAL_VARS].copy()

# ----------------------------
# Imputation simple et propre
# ----------------------------

# Numériques → médiane
for col in NUM_VARS:
    med = X[col].median()
    X[col] = pd.to_numeric(X[col], errors="coerce").fillna(med)

# Catégorielles → "__MISSING__"
for col in CAT_VARS:
    X[col] = (
        X[col]
        .astype("object")
        .where(~X[col].isna(), "__MISSING__")
        .astype(str)
    )

# ----------------------------
# Vérifications finales
# ----------------------------
assert X.isna().sum().sum() == 0, "NaN restants dans X"
assert set(y.unique()).issubset({0, 1}), "Cible non binaire"

print("[OK] Imputation terminée")
print("[OK] Variables finales :", X.shape[1])

# ----------------------------
# Exports
# ----------------------------
X.to_csv(os.path.join(OUT_DIR, "X_ml_final.csv"), index=False)
y.to_csv(os.path.join(OUT_DIR, "y.csv"), index=False)
w.to_csv(os.path.join(OUT_DIR, "weights.csv"), index=False)

print("\n[OK] Exports terminés :")
print(" - X_ml_final.csv")
print(" - y.csv")
print(" - weights.csv")


# Identitifcation et reconstruction des couples

In [10]:
"""
CODE 1 — Reconstruction des couples (matching STRICT LPRM=1/2)
Stratégie validée (Picard) :
- ID_ZONE = COMMUNE si NIVEAU_APPARIEMENT='1_DEPCOM' sinon CANTVILLE
- ID_FAMILLE = ID_ZONE + NUMMI + NUMF  (SANS restriction sur NUMF)
- Candidats = individus avec LPRM ∈ {1,2} et NUMMI != 'Z'
- Couples sûrs = ID_FAMILLE avec exactement 1 LPRM=1 et 1 LPRM=2
Sorties :
- CSV "long" : 2 lignes par couple, avec ID_COUPLE = ID_FAMILLE
- Excel sample 10k lignes pour contrôle visuel
- Diagnostics console : tailles, patterns LPRM, taux de reconstruction, audits NA/collisions

Auteur : Projet INSEE — Mobilité Transfrontalière
Date : Janvier 2026
"""

import pandas as pd
import numpy as np
import time

# ============================================================
# PARAMÈTRES (adapte tes chemins)
# ============================================================
INPUT_PATH = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/BDD_FINAL_CLEAN.csv"

OUT_COUPLES_LONG_CSV = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/COUPLES_LONG_MATCHED.csv"
OUT_SAMPLE_XLSX      = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/COUPLES_LONG_sample10k.xlsx"

SEP = ";"  # ton fichier est en ;
ENC = "utf-8"

SAMPLE_N = 10_000
RANDOM_STATE = 42

# ============================================================
# HELPERS
# ============================================================
def pct(x, n):
    return 100 * x / n if n else 0.0

def tprint(msg):
    print(msg, flush=True)

# ============================================================
# 0) START
# ============================================================
t0 = time.time()
tprint("=" * 85)
tprint("CODE 1 — MATCHING COUPLES (LPRM=1/2) — COMMUNE/CANTVILLE + NUMMI + NUMF")
tprint("=" * 85)

# ============================================================
# 1) CHARGEMENT
# ============================================================
tprint("\n[1] Chargement CSV …")
df = pd.read_csv(INPUT_PATH, sep=SEP, encoding=ENC, low_memory=False)
tprint(f"OK: {df.shape[0]:,} lignes | {df.shape[1]} colonnes")
tprint("Colonnes clés attendues : COMMUNE, CANTVILLE, NIVEAU_APPARIEMENT, NUMMI, NUMF, LPRM")

required = ["COMMUNE", "CANTVILLE", "NIVEAU_APPARIEMENT", "NUMMI", "NUMF", "LPRM"]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"Colonnes manquantes: {missing}")

# ============================================================
# 2) NORMALISATION DES TYPES (CRITIQUE)
# ============================================================
tprint("\n[2] Normalisation types …")

# NUMMI / NUMF sont object avec valeurs 'Z' possibles (cf dictionnaire)
df["NUMMI"] = df["NUMMI"].astype("string").str.strip().str.upper()
df["NUMF"]  = df["NUMF"].astype("string").str.strip().str.upper()

# LPRM est float avec manquants ~0.63% : on force Int64 propre
df["LPRM"] = pd.to_numeric(df["LPRM"], errors="coerce").astype("Int64")

# COMMUNE / CANTVILLE int64 -> on garde, mais on sécurise NA (normalement 0% chez toi)
# On convertit en Int64 pour pouvoir détecter NA si jamais un jour ça arrive.
df["COMMUNE"]   = pd.to_numeric(df["COMMUNE"], errors="coerce").astype("Int64")
df["CANTVILLE"] = pd.to_numeric(df["CANTVILLE"], errors="coerce").astype("Int64")

# Niveau appariement
df["NIVEAU_APPARIEMENT"] = df["NIVEAU_APPARIEMENT"].astype("string").str.strip()

# ============================================================
# 3) AUDIT NA / CAS IMPOSSIBLES
# ============================================================
tprint("\n[3] Audit NA / cohérence minimale …")

na_commune = df["COMMUNE"].isna().sum()
na_cant    = df["CANTVILLE"].isna().sum()
na_nummi   = df["NUMMI"].isna().sum()
na_numf    = df["NUMF"].isna().sum()
na_lprm    = df["LPRM"].isna().sum()

tprint(f"NA COMMUNE   : {na_commune:,}")
tprint(f"NA CANTVILLE : {na_cant:,}")
tprint(f"NA NUMMI     : {na_nummi:,}")
tprint(f"NA NUMF      : {na_numf:,}")
tprint(f"NA LPRM      : {na_lprm:,}")

# Cas incohérents (NIVEAU_APPARIEMENT demande COMMUNE mais COMMUNE manquante, etc.)
mask_need_commune = (df["NIVEAU_APPARIEMENT"] == "1_DEPCOM") & (df["COMMUNE"].isna())
mask_need_cant    = (df["NIVEAU_APPARIEMENT"] == "2_CANTVILLE") & (df["CANTVILLE"].isna())

tprint(f"Incohérences: 1_DEPCOM mais COMMUNE NA     : {mask_need_commune.sum():,}")
tprint(f"Incohérences: 2_CANTVILLE mais CANTVILLE NA: {mask_need_cant.sum():,}")

# ============================================================
# 4) CONSTRUCTION ID_ZONE puis ID_FAMILLE
# ============================================================
tprint("\n[4] Construction ID_ZONE + ID_FAMILLE …")

# ID_ZONE dépend du niveau d'appariement
id_zone = np.where(
    df["NIVEAU_APPARIEMENT"].to_numpy() == "1_DEPCOM",
    "COM_" + df["COMMUNE"].astype("string").to_numpy(),
    "CAN_" + df["CANTVILLE"].astype("string").to_numpy()
)

# ID_FAMILLE = zone + NUMMI + NUMF (sans restriction sur NUMF)
df["ID_FAMILLE"] = (
    pd.Series(id_zone, index=df.index).astype("string")
    + "_M" + df["NUMMI"]
    + "_F" + df["NUMF"]
)

tprint("OK: ID_FAMILLE créée")
tprint(f"Nb ID_FAMILLE distincts (toute base): {df['ID_FAMILLE'].nunique():,}")

# ============================================================
# 5) CANDIDATS COUPLE (LPRM in {1,2} & NUMMI != 'Z')
# ============================================================
tprint("\n[5] Sélection candidats (LPRM ∈ {1,2} & NUMMI != 'Z') …")

cand = df[(df["LPRM"].isin([1, 2])) & (df["NUMMI"] != "Z")].copy()

tprint(f"Candidats: {len(cand):,} individus")
tprint(f"  dont LPRM=1 : {(cand['LPRM'] == 1).sum():,}")
tprint(f"  dont LPRM=2 : {(cand['LPRM'] == 2).sum():,}")

# ============================================================
# 6) DIAGNOSTICS STRUCTURE AVANT MATCHING
# ============================================================
tprint("\n[6] Diagnostics avant matching …")

# Taille du groupe (sur candidats) par ID_FAMILLE
sizes = cand.groupby("ID_FAMILLE").size()
tprint("Taille des groupes (candidats) — top fréquences :")
print(sizes.value_counts().sort_index().head(10).to_string())

# Pattern LPRM (comptes de 1 et 2 par famille) — vectorisé
tab = pd.crosstab(cand["ID_FAMILLE"], cand["LPRM"])
if 1 not in tab.columns: tab[1] = 0
if 2 not in tab.columns: tab[2] = 0

# Quelques patterns clés
n_ids = tab.shape[0]
pattern_11 = ((tab[1] == 1) & (tab[2] == 1)).sum()
pattern_10 = ((tab[1] == 1) & (tab[2] == 0)).sum()
pattern_01 = ((tab[1] == 0) & (tab[2] == 1)).sum()
pattern_12p = ((tab[1] == 1) & (tab[2] >= 2)).sum()
pattern_21p = ((tab[1] >= 2) & (tab[2] == 1)).sum()
pattern_22p = ((tab[1] >= 2) & (tab[2] >= 2)).sum()

tprint("\nPatterns LPRM par ID_FAMILLE (sur candidats) :")
tprint(f"  (1,1) exact  -> couples sûrs potentiels : {pattern_11:,} ({pct(pattern_11, n_ids):.2f}%)")
tprint(f"  (1,0)        -> ref sans conjoint      : {pattern_10:,} ({pct(pattern_10, n_ids):.2f}%)")
tprint(f"  (0,1)        -> conjoint sans ref      : {pattern_01:,} ({pct(pattern_01, n_ids):.2f}%)")
tprint(f"  (1,>=2)      -> 1 ref / multi conj     : {pattern_12p:,} ({pct(pattern_12p, n_ids):.2f}%)")
tprint(f"  (>=2,1)      -> multi ref / 1 conj     : {pattern_21p:,} ({pct(pattern_21p, n_ids):.2f}%)")
tprint(f"  (>=2,>=2)    -> multi/multi            : {pattern_22p:,} ({pct(pattern_22p, n_ids):.2f}%)")

# ============================================================
# 7) MATCHING STRICT : exactement 1 LPRM=1 et 1 LPRM=2
# ============================================================
tprint("\n[7] Matching strict (1 ref + 1 conjoint) …")

ok_ids = tab.index[(tab[1] == 1) & (tab[2] == 1)]
tprint(f"ID_FAMILLE matchés (couples sûrs) : {len(ok_ids):,}")

couples_long = cand[cand["ID_FAMILLE"].isin(ok_ids)].copy()

# Sécurité : il doit y avoir exactement 2 lignes par ID_FAMILLE
check_sizes = couples_long.groupby("ID_FAMILLE").size()
bad = (check_sizes != 2).sum()
if bad:
    tprint(f"⚠️ ATTENTION: {bad:,} familles n'ont pas exactement 2 lignes après filtrage (devrait être 0)")
else:
    tprint("OK: 2 lignes par couple (contrôle passé)")

# ID_COUPLE = ID_FAMILLE
couples_long["ID_COUPLE"] = couples_long["ID_FAMILLE"]

# ROLE_COUPLE pour lisibilité
couples_long["ROLE_COUPLE"] = np.where(couples_long["LPRM"] == 1, "REF", "CONJ")

tprint(f"Couples (long) : {len(couples_long):,} lignes -> {couples_long['ID_COUPLE'].nunique():,} couples")

# ============================================================
# 8) PETITS CHECKS QUALITATIFS RAPIDES (sans score, ça c'est Code 2)
# ============================================================
tprint("\n[8] Checks rapides …")

# Si COUPLE existe, on regarde la cohérence déclarative sans filtrer ici
if "COUPLE" in couples_long.columns:
    tmp = pd.crosstab(couples_long["ROLE_COUPLE"], couples_long["COUPLE"], margins=True)
    tprint("Table ROLE_COUPLE × COUPLE (sur couples matchés) :")
    print(tmp.to_string())

# Distribution NUMF dans les couples (pour vérifier que tu n'as pas “involontairement” restreint)
tprint("\nDistribution NUMF (sur couples matchés) :")
print(couples_long["NUMF"].value_counts(dropna=False).head(10).to_string())

# ============================================================
# 9) EXPORT CSV COUPLES LONG
# ============================================================
tprint("\n[9] Export CSV couples (long) …")
# On met devant les colonnes utiles
front_cols = ["ID_COUPLE", "ID_FAMILLE", "ROLE_COUPLE", "LPRM",
              "NIVEAU_APPARIEMENT", "COMMUNE", "CANTVILLE", "NUMMI", "NUMF"]
front_cols = [c for c in front_cols if c in couples_long.columns]
cols = front_cols + [c for c in couples_long.columns if c not in front_cols]

couples_long[cols].to_csv(OUT_COUPLES_LONG_CSV, index=False, sep=SEP, encoding=ENC)
tprint(f"OK: couples exportés -> {OUT_COUPLES_LONG_CSV}")
tprint(f"    {couples_long['ID_COUPLE'].nunique():,} couples | {len(couples_long):,} lignes")

# ============================================================
# 10) SAMPLE EXCEL 10k (contrôle visuel)
# ============================================================
tprint("\n[10] Export Excel sample 10k lignes …")

sample = couples_long.sample(n=min(SAMPLE_N, len(couples_long)), random_state=RANDOM_STATE).copy()
front2 = ["ID_COUPLE", "ROLE_COUPLE", "LPRM", "SEXE", "AGEREV", "DIPL", "ILT", "COUPLE",
          "TYPMR", "TDM8", "MOCO_DET", "STAT_CONJ",
          "NIVEAU_APPARIEMENT", "COMMUNE", "CANTVILLE", "NUMMI", "NUMF", "DEPT", "DEPCOM"]
front2 = [c for c in front2 if c in sample.columns]
sample = sample[front2 + [c for c in sample.columns if c not in front2]]

sample.to_excel(OUT_SAMPLE_XLSX, index=False)
tprint(f"OK: sample Excel -> {OUT_SAMPLE_XLSX}")

# ============================================================
# 11) FIN
# ============================================================
dt = time.time() - t0
tprint("\n" + "=" * 85)
tprint(f"TERMINÉ — temps total: {dt:.1f}s")
tprint("=" * 85)


CODE 1 — MATCHING COUPLES (LPRM=1/2) — COMMUNE/CANTVILLE + NUMMI + NUMF



[1] Chargement CSV …


KeyboardInterrupt: 

# Validation du matching 

In [3]:
import pandas as pd
import numpy as np
import time

t0 = time.time()

# ============================================================
# PARAMÈTRES
# ============================================================
INPUT_COUPLES_LONG = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/COUPLES_LONG_MATCHED.csv"

OUT_LONG_ENRICHED = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/COUPLES_LONG_VALIDATED.csv"
OUT_WIDE = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/COUPLES_WIDE_VALIDATED.csv"
OUT_SAMPLE_XLSX = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/COUPLES_VALIDATED_sample10k.xlsx"

SAMPLE_N = 10_000
RANDOM_STATE = 42

# ---- PARAMS SCORE (à ajuster seulement si tu veux) ----
TYPMR_COUPLE_SET = {41, 42, 43, 44}          # selon ton plan (classique)
TDM8_COUPLE_SET = {200, 302, 310, 311}       # selon ton plan (classique)
MOCO_ADULTE_SET = {"A0", "A1", "A2", "A3"}    # selon ton plan (adulte famille)

# STAT_CONJ : INCONNU -> on met par défaut {1,2,3} MAIS on imprimera la distribution.
STAT_CONJ_UNION_SET = {1, 2, 3}

# ============================================================
# 1) CHARGEMENT
# ============================================================
print("=" * 100)
print("CODE 2 — VALIDATION / COHÉRENCE DES COUPLES (LONG -> LONG enrichi + WIDE)")
print("=" * 100)

print("\n[1] Chargement …")
df = pd.read_csv(INPUT_COUPLES_LONG, sep=";", encoding="utf-8", low_memory=False)
print(f"OK: {df.shape[0]:,} lignes | {df.shape[1]} colonnes")
print("Colonnes présentes (extrait):", list(df.columns[:15]))

required = ["ID_COUPLE", "ROLE_COUPLE", "LPRM"]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"Colonnes manquantes: {missing}")

# ============================================================
# 2) NORMALISATION TYPES
# ============================================================
print("\n[2] Normalisation types …")

df["ROLE_COUPLE"] = df["ROLE_COUPLE"].astype("string").str.strip().str.upper()

# Numériques (robustes)
for c in ["COUPLE", "TYPMR", "STAT_CONJ", "AGEREV", "AGED", "SEXE", "ILT", "IPONDI"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# TDM8 est catégorielle avec 'ZZZ' -> on crée une version numérique dédiée
if "TDM8" in df.columns:
    df["TDM8_NUM"] = pd.to_numeric(df["TDM8"].astype("string").str.replace("ZZZ", "", regex=False), errors="coerce")

# Strings
for c in ["MOCO_DET", "NUMF", "NUMMI", "NIVEAU_APPARIEMENT", "DCFLT", "WORK_IN_PAYS", "SFM", "PCSL", "NA5"]:
    if c in df.columns:
        df[c] = df[c].astype("string").str.strip().str.upper()

print("OK: types normalisés.")

# ============================================================
# 3) CHECK STRUCTUREL (doit être parfait après ton Code 1)
# ============================================================
print("\n[3] Checks structurels …")

gsize = df.groupby("ID_COUPLE").size()
bad_size = (gsize != 2).sum()
print(f"Couples avec taille != 2: {bad_size:,} (attendu 0)")

role_tab = df.pivot_table(index="ID_COUPLE", columns="ROLE_COUPLE", values="LPRM", aggfunc="size", fill_value=0)
for col in ["REF", "CONJ"]:
    if col not in role_tab.columns:
        role_tab[col] = 0

bad_role = ((role_tab["REF"] != 1) | (role_tab["CONJ"] != 1)).sum()
print(f"Couples sans exactement 1 REF et 1 CONJ: {bad_role:,} (attendu 0)")

if bad_size > 0 or bad_role > 0:
    print("⚠️ WARNING: structure non parfaite -> ton fichier d’entrée n’est pas celui du Code 1 strict.")
    # On continue quand même pour diagnostiquer.

n_couples = df["ID_COUPLE"].nunique()
print(f"Nb couples: {n_couples:,}")

# ============================================================
# 4) CONSTRUIRE UNE TABLE COUPLE-LEVEL (REF/CONJ côte à côte)
#    -> c’est PLUS PROPRE pour scorer et diagnostiquer
# ============================================================
print("\n[4] Construction table couple-level (REF/CONJ côte à côte) …")

vars_for_checks = []
for v in ["COUPLE", "TYPMR", "TDM8_NUM", "STAT_CONJ", "MOCO_DET", "AGEREV", "ILT", "NUMF", "NIVEAU_APPARIEMENT"]:
    if v in df.columns:
        vars_for_checks.append(v)

wide_chk = df.pivot_table(index="ID_COUPLE", columns="ROLE_COUPLE", values=vars_for_checks, aggfunc="first")
wide_chk.columns = [f"{v}_{r}" for (v, r) in wide_chk.columns]
wide_chk = wide_chk.reset_index()

print(f"OK: couple-level -> {wide_chk.shape[0]:,} lignes | {wide_chk.shape[1]} colonnes")

# ============================================================
# 5) DIAGNOSTICS BRUTS (avant score)
# ============================================================
print("\n[5] Diagnostics bruts …")

# COUPLE incohérences
if "COUPLE_REF" in wide_chk.columns and "COUPLE_CONJ" in wide_chk.columns:
    both1 = ((wide_chk["COUPLE_REF"] == 1) & (wide_chk["COUPLE_CONJ"] == 1)).mean()
    any_not1 = ((wide_chk["COUPLE_REF"] != 1) | (wide_chk["COUPLE_CONJ"] != 1)).sum()
    print(f"COUPLE déclaratif parfait (1 & 1): {both1*100:.2f}%")
    print(f"Couples avec au moins 1 membre COUPLE!=1: {any_not1:,} ({any_not1/n_couples*100:.2f}%)")

# NUMF bizarre (tu as vu 18 'Z')
if "NUMF_REF" in wide_chk.columns:
    print("\nTop modalités NUMF_REF:")
    print(wide_chk["NUMF_REF"].value_counts(dropna=False).head(5))

# STAT_CONJ distribution pour que tu confirmes le mapping
if "STAT_CONJ_REF" in wide_chk.columns:
    print("\nDistribution STAT_CONJ (REF) :")
    print(wide_chk["STAT_CONJ_REF"].value_counts(dropna=False).sort_index())

if "STAT_CONJ_CONJ" in wide_chk.columns:
    print("\nDistribution STAT_CONJ (CONJ) :")
    print(wide_chk["STAT_CONJ_CONJ"].value_counts(dropna=False).sort_index())

# ============================================================
# 6) SCORE_QUALITE (0–5) + FLAGS détaillés
#    Q1: les deux COUPLE==1
#    Q2: TYPMR dans set (au moins un des deux, typiquement identique)
#    Q3: TDM8 dans set (au moins un)
#    Q4: au moins un STAT_CONJ dans set union
#    Q5: les deux MOCO_DET dans set adulte
# ============================================================
print("\n[6] Calcul SCORE_QUALITE (0–5) …")

# Q1
if "COUPLE_REF" in wide_chk.columns and "COUPLE_CONJ" in wide_chk.columns:
    wide_chk["Q1_COUPLE_11"] = ((wide_chk["COUPLE_REF"] == 1) & (wide_chk["COUPLE_CONJ"] == 1)).astype("int8")
else:
    wide_chk["Q1_COUPLE_11"] = 0

# Q2
if "TYPMR_REF" in wide_chk.columns:
    wide_chk["Q2_TYPMR"] = wide_chk["TYPMR_REF"].isin(TYPMR_COUPLE_SET).astype("int8")
elif "TYPMR_CONJ" in wide_chk.columns:
    wide_chk["Q2_TYPMR"] = wide_chk["TYPMR_CONJ"].isin(TYPMR_COUPLE_SET).astype("int8")
else:
    wide_chk["Q2_TYPMR"] = 0

# Q3
if "TDM8_NUM_REF" in wide_chk.columns:
    wide_chk["Q3_TDM8"] = wide_chk["TDM8_NUM_REF"].isin(TDM8_COUPLE_SET).astype("int8")
elif "TDM8_NUM_CONJ" in wide_chk.columns:
    wide_chk["Q3_TDM8"] = wide_chk["TDM8_NUM_CONJ"].isin(TDM8_COUPLE_SET).astype("int8")
else:
    wide_chk["Q3_TDM8"] = 0

# Q4 (STAT_CONJ -> union)
if "STAT_CONJ_REF" in wide_chk.columns and "STAT_CONJ_CONJ" in wide_chk.columns:
    wide_chk["Q4_STAT_CONJ"] = (
        wide_chk["STAT_CONJ_REF"].isin(STAT_CONJ_UNION_SET) |
        wide_chk["STAT_CONJ_CONJ"].isin(STAT_CONJ_UNION_SET)
    ).astype("int8")
else:
    wide_chk["Q4_STAT_CONJ"] = 0

# Q5 MOCO_DET (les deux)
if "MOCO_DET_REF" in wide_chk.columns and "MOCO_DET_CONJ" in wide_chk.columns:
    wide_chk["Q5_MOCO"] = (
        wide_chk["MOCO_DET_REF"].isin(MOCO_ADULTE_SET) &
        wide_chk["MOCO_DET_CONJ"].isin(MOCO_ADULTE_SET)
    ).astype("int8")
else:
    wide_chk["Q5_MOCO"] = 0

wide_chk["SCORE_QUALITE"] = (
    wide_chk["Q1_COUPLE_11"] +
    wide_chk["Q2_TYPMR"] +
    wide_chk["Q3_TDM8"] +
    wide_chk["Q4_STAT_CONJ"] +
    wide_chk["Q5_MOCO"]
).astype("int8")

print("Distribution SCORE_QUALITE :")
print(wide_chk["SCORE_QUALITE"].value_counts().sort_index())
print(f"Score moyen: {wide_chk['SCORE_QUALITE'].mean():.2f} / 5")

# ============================================================
# 7) AJOUT VARIABLES PROBIT bivarié (Y_REF / Y_CONJ) + écarts
# ============================================================
print("\n[7] Variables analytiques (Y_REF/Y_CONJ + écart âge) …")

if "ILT_REF" in wide_chk.columns:
    wide_chk["Y_REF"] = (wide_chk["ILT_REF"] == 7).astype("int8")
if "ILT_CONJ" in wide_chk.columns:
    wide_chk["Y_CONJ"] = (wide_chk["ILT_CONJ"] == 7).astype("int8")

if "AGEREV_REF" in wide_chk.columns and "AGEREV_CONJ" in wide_chk.columns:
    wide_chk["ECART_AGE_AGEREV"] = (wide_chk["AGEREV_REF"] - wide_chk["AGEREV_CONJ"]).abs()

if "Y_REF" in wide_chk.columns and "Y_CONJ" in wide_chk.columns:
    ct = pd.crosstab(wide_chk["Y_REF"], wide_chk["Y_CONJ"], margins=True)
    print("\nTable décision transfrontalière (ILT==7):")
    print(ct)

# ============================================================
# 8) MERGE SCORE DANS LE LONG + EXPORTS
# ============================================================
print("\n[8] Merge vers LONG + exports …")

df = df.merge(
    wide_chk[["ID_COUPLE", "SCORE_QUALITE", "Q1_COUPLE_11", "Q2_TYPMR", "Q3_TDM8", "Q4_STAT_CONJ", "Q5_MOCO"]]
    , on="ID_COUPLE", how="left"
)

# Export LONG enrichi
df.to_csv(OUT_LONG_ENRICHED, sep=";", index=False, encoding="utf-8")
print(f"OK: LONG enrichi -> {OUT_LONG_ENRICHED}")

# Export WIDE validé
wide_chk.to_csv(OUT_WIDE, sep=";", index=False, encoding="utf-8")
print(f"OK: WIDE validé -> {OUT_WIDE}")

# Sample Excel (WIDE)
sample = wide_chk.sample(n=min(SAMPLE_N, len(wide_chk)), random_state=RANDOM_STATE).copy()
front = ["ID_COUPLE", "SCORE_QUALITE", "Y_REF", "Y_CONJ", "ECART_AGE_AGEREV",
         "COUPLE_REF", "COUPLE_CONJ", "STAT_CONJ_REF", "STAT_CONJ_CONJ",
         "TYPMR_REF", "TDM8_NUM_REF", "MOCO_DET_REF", "MOCO_DET_CONJ",
         "NIVEAU_APPARIEMENT_REF", "NUMF_REF"]
front = [c for c in front if c in sample.columns]
sample = sample[front + [c for c in sample.columns if c not in front]]
sample.to_excel(OUT_SAMPLE_XLSX, index=False)
print(f"OK: Sample Excel -> {OUT_SAMPLE_XLSX}")

dt = time.time() - t0
print("\n" + "=" * 100)
print(f"TERMINÉ — temps total: {dt:.1f}s")
print("=" * 100)


CODE 2 — VALIDATION / COHÉRENCE DES COUPLES (LONG -> LONG enrichi + WIDE)

[1] Chargement …
OK: 253,948 lignes | 109 colonnes
Colonnes présentes (extrait): ['ID_COUPLE', 'ID_FAMILLE', 'ROLE_COUPLE', 'LPRM', 'NIVEAU_APPARIEMENT', 'COMMUNE', 'CANTVILLE', 'NUMMI', 'NUMF', 'ARM', 'DCFLT', 'DCLT', 'AGEREVQ', 'GS', 'DEROU']

[2] Normalisation types …
OK: types normalisés.

[3] Checks structurels …
Couples avec taille != 2: 0 (attendu 0)
Couples sans exactement 1 REF et 1 CONJ: 0 (attendu 0)
Nb couples: 126,974

[4] Construction table couple-level (REF/CONJ côte à côte) …
OK: couple-level -> 126,974 lignes | 19 colonnes

[5] Diagnostics bruts …
COUPLE déclaratif parfait (1 & 1): 99.55%
Couples avec au moins 1 membre COUPLE!=1: 574 (0.45%)

Top modalités NUMF_REF:
NUMF_REF
1    126965
Z         9
Name: count, dtype: Int64

Distribution STAT_CONJ (REF) :
STAT_CONJ_REF
1    75123
2    21981
3    28703
4       35
5      305
6      827
Name: count, dtype: int64

Distribution STAT_CONJ (CONJ) :
STA